In [ ]:
!unzip /content/drive/MyDrive/train.zip -d /content/train

In [2]:
%%capture
!git clone https://github.com/huggingface/diffusers
%cd diffusers
!pip install .

In [3]:
%%capture
%cd /content/diffusers/examples/unconditional_image_generation
!pip install -r requirements.txt

In [ ]:
!pip install accelerate

In [ ]:
!accelerate config default
!mv /content/default_config.yaml /root/.cache/huggingface/accelerate/
!accelerate config update

In [6]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
import os
import glob
import shutil
import pandas as pd

In [18]:
files = glob.glob("/content/train/*/*/*.webp")
os.makedirs("/content/train_data", exist_ok=True)

for idx, f in enumerate(files):
    shutil.copy2(f, f"/content/train_data/{idx}.webp")

In [17]:
len(glob.glob("/content/train_data/*.webp"))

120

In [16]:
!accelerate launch --mixed_precision="fp16" train_unconditional.py \
  --train_data_dir="/content/train_data" \
  --resolution=64 --random_flip --center_crop \
  --train_batch_size=16 \
  --num_epochs=100 --checkpointing_steps=5000 \
  --learning_rate=1e-04 --lr_scheduler="constant" --lr_warmup_steps=500 \
  --output_dir="/content/output" 

2023-05-02 08:45:46.045258: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-02 08:45:50.761871: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:249: FutureWarning: `logging_dir` is deprecated and will be removed in version 0.18.0 of 🤗 Accelerate. Use `project_dir` instead.
  warnings.warn(
05/02/2023 08:45:53 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: no

Resolving data files: 100% 120/120 [00:00<00:00, 98612.16it/s]
05/02/2023 08:45:54 - WARNING - datasets.builder - Found cached dataset imagefolder (/root/.cache/huggingface/datasets/imagefolder/default-19a6e92df275680b/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)
05/02/2023 08:45:54 - INFO - __main__ - Dataset size: 120
05/02/2023 08:45:54 - INF

In [11]:
!cp -r /content/output -d /content/drive/MyDrive/hse-new

In [13]:
from diffusers import DDIMPipeline, DDIMScheduler

device = "cuda"

# load model and scheduler
pipeline = DDIMPipeline.from_pretrained('/content/output', local_files_only=True)
pipeline.scheduler = DDIMScheduler.from_config(pipeline.scheduler.config)
pipeline.to(device)

# generate image
image = pipeline(num_inference_steps=50).images[0]
# save image
image.save("/content/image.png")

  0%|          | 0/50 [00:00<?, ?it/s]